In [1]:
from IPython.display import Image
# library
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import math
import statistics
import seaborn as sns
from IPython.display import display
import sklearn

%matplotlib inline

# 特徵選取會用到的函數
from sklearn.feature_selection import VarianceThreshold
from sklearn import preprocessing

In [2]:
# 特徵工程：基於原始資料中萃取出好特徵，藉此改善模型性能的過程。
# 好的特徵，具備以下三種特性：
    #變化性
    #預測性
    #辨識力

In [3]:
# 特徵選取的三大方法
 # 過濾法(Filter) :列入一些篩選特徵的標準，把具變化性以及與目標變數相關的特徵，挑選出具變化性以及中高度相關的特徵。
        #移除低變異數的特徵，透過設定門檻，來移除低變異數特徵
        #變量特徵選取（Univariate Feature Selection）
            #目標變數為離散型，採用卡方檢定(chi2)
            #目標變數為連續型，可採用 f_regression
 # 裝法(Wrapper)
 #嵌入法(Embedded):使用機器學習或模型進行訓練，得到某些特徵的權重係數，根據係數的重要性來選擇，類似過濾法，但採用訓練結果來選擇特徵。


In [4]:
#用字典產生一組資料
data={'sex': ['Male','Male','Male','Male','Male','Female','Female','Female','Female','Female','Male','Male','Male','Male','Male','Female','Female','Female','Female','Female'],
      'insomnia':['Y','N','N','N','N','N','Y','Y','Y','N','Y','N','N','N','N','N','Y','Y','Y','N'],
     'age':[23,40,5,30,1,40,16,27,43,8,23,39,5,29,1,42,13,29,41,10],
     'height':[180,170,100,176,70,160,170,166,155,35,170,168,101,175,72,163,169,163,151,40],
     'weight':[100,68,20,70,10,45,50,58,58,17,101,65,22,79,12,40,53,52,56,14]}
#轉成 dataframe格式 
data=pd.DataFrame(data)
display(data)

print(data.info())


,sex,insomnia,age,height,weight
0,Male,Y,23,180,100
1,Male,N,40,170,68
2,Male,N,5,100,20
3,Male,N,30,176,70
4,Male,N,1,70,10
5,Female,N,40,160,45
6,Female,Y,16,170,50
7,Female,Y,27,166,58
8,Female,Y,43,155,58
9,Female,N,8,35,17


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sex       20 non-null     object
 1   insomnia  20 non-null     object
 2   age       20 non-null     int64 
 3   height    20 non-null     int64 
 4   weight    20 non-null     int64 
dtypes: int64(3), object(2)
memory usage: 928.0+ bytes
None


In [5]:
num_features = []
for dtype, feature in zip(data.dtypes, data.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

3 Numeric Features : ['age', 'height', 'weight']



In [6]:
cat_features = []
for dtype, feature in zip(data.dtypes, data.columns):
    if dtype == 'object':
        cat_features.append(feature)
print(f'{len(cat_features)} category Features : {cat_features}\n')

2 category Features : ['sex', 'insomnia']



In [7]:
# 過濾法：
    # Step0：標準化(最大最小值)
    #
    # Step1：運用 VarianceThreshold 設定門檻
filter1=VarianceThreshold(threshold=(100))#以 變異數為 100 為臨界值，超過的特徵留下，小於的特徵去除。
    # Step2：透過函數做計算，過濾
data_filter1= filter1.fit_transform(data[num_features])

    # Step3：確定哪一些特徵留下來
display(data_filter1)    


print("Variances is %s" % filter1.variances_) #計算每一個特徵的變異數 
print("The surport is %s" % filter1.get_support(True)) #確定那些特徵符合條件留下

array([[ 23, 180, 100],
       [ 40, 170,  68],
       [  5, 100,  20],
       [ 30, 176,  70],
       [  1,  70,  10],
       [ 40, 160,  45],
       [ 16, 170,  50],
       [ 27, 166,  58],
       [ 43, 155,  58],
       [  8,  35,  17],
       [ 23, 170, 101],
       [ 39, 168,  65],
       [  5, 101,  22],
       [ 29, 175,  79],
       [  1,  72,  12],
       [ 42, 163,  40],
       [ 13, 169,  53],
       [ 29, 163,  52],
       [ 41, 151,  56],
       [ 10,  40,  14]])

Variances is [ 210.6875 2224.51    715.05  ]
The surport is [0 1 2]


In [8]:
 # 過濾法：
    # Step0：標準化(最大最小值)
    #先標準化，才能用統一標準篩選，z = (x - u) / s
    #std=preprocessing.StandardScaler()
    #不能用標準化，因為這樣的變異數都是 1 ~
std=preprocessing.MinMaxScaler()
data_std=std.fit_transform(data[num_features])
display(data_std)

    # Step1：運用 VarianceThreshold 設定門檻
    #是初步篩選，不要太嚴格
filter1=VarianceThreshold(threshold=(0.05))

    # Step2：透過函數做計算，過濾
data_filter1= filter1.fit_transform(data_std)


    # Step3：確定哪一些特徵留下來
display(data_filter1)
    
print("Variances is %s" % filter1.variances_)
print("After transform is %s" % filter1.transform(data_std))
print("The surport is %s" % filter1.get_support(True)) #確定那些變數符合條件留下
print("After reverse transform is %s" %filter1.inverse_transform(filter1.transform(data_std)))

array([[0.52380952, 1.        , 0.98901099],
       [0.92857143, 0.93103448, 0.63736264],
       [0.0952381 , 0.44827586, 0.10989011],
       [0.69047619, 0.97241379, 0.65934066],
       [0.        , 0.24137931, 0.        ],
       [0.92857143, 0.86206897, 0.38461538],
       [0.35714286, 0.93103448, 0.43956044],
       [0.61904762, 0.90344828, 0.52747253],
       [1.        , 0.82758621, 0.52747253],
       [0.16666667, 0.        , 0.07692308],
       [0.52380952, 0.93103448, 1.        ],
       [0.9047619 , 0.91724138, 0.6043956 ],
       [0.0952381 , 0.45517241, 0.13186813],
       [0.66666667, 0.96551724, 0.75824176],
       [0.        , 0.25517241, 0.02197802],
       [0.97619048, 0.88275862, 0.32967033],
       [0.28571429, 0.92413793, 0.47252747],
       [0.66666667, 0.88275862, 0.46153846],
       [0.95238095, 0.8       , 0.50549451],
       [0.21428571, 0.03448276, 0.04395604]])

array([[0.52380952, 1.        , 0.98901099],
       [0.92857143, 0.93103448, 0.63736264],
       [0.0952381 , 0.44827586, 0.10989011],
       [0.69047619, 0.97241379, 0.65934066],
       [0.        , 0.24137931, 0.        ],
       [0.92857143, 0.86206897, 0.38461538],
       [0.35714286, 0.93103448, 0.43956044],
       [0.61904762, 0.90344828, 0.52747253],
       [1.        , 0.82758621, 0.52747253],
       [0.16666667, 0.        , 0.07692308],
       [0.52380952, 0.93103448, 1.        ],
       [0.9047619 , 0.91724138, 0.6043956 ],
       [0.0952381 , 0.45517241, 0.13186813],
       [0.66666667, 0.96551724, 0.75824176],
       [0.        , 0.25517241, 0.02197802],
       [0.97619048, 0.88275862, 0.32967033],
       [0.28571429, 0.92413793, 0.47252747],
       [0.66666667, 0.88275862, 0.46153846],
       [0.95238095, 0.8       , 0.50549451],
       [0.21428571, 0.03448276, 0.04395604]])

Variances is [0.11943736 0.10580309 0.08634827]
After transform is [[0.52380952 1.         0.98901099]
 [0.92857143 0.93103448 0.63736264]
 [0.0952381  0.44827586 0.10989011]
 [0.69047619 0.97241379 0.65934066]
 [0.         0.24137931 0.        ]
 [0.92857143 0.86206897 0.38461538]
 [0.35714286 0.93103448 0.43956044]
 [0.61904762 0.90344828 0.52747253]
 [1.         0.82758621 0.52747253]
 [0.16666667 0.         0.07692308]
 [0.52380952 0.93103448 1.        ]
 [0.9047619  0.91724138 0.6043956 ]
 [0.0952381  0.45517241 0.13186813]
 [0.66666667 0.96551724 0.75824176]
 [0.         0.25517241 0.02197802]
 [0.97619048 0.88275862 0.32967033]
 [0.28571429 0.92413793 0.47252747]
 [0.66666667 0.88275862 0.46153846]
 [0.95238095 0.8        0.50549451]
 [0.21428571 0.03448276 0.04395604]]
The surport is [0 1 2]
After reverse transform is [[0.52380952 1.         0.98901099]
 [0.92857143 0.93103448 0.63736264]
 [0.0952381  0.44827586 0.10989011]
 [0.69047619 0.97241379 0.65934066]
 [0.         0.241

In [9]:
# 單變量特徵選取
    # Step1：根據目標變量是連續或離散，來決定判斷的準則。

    # Step2：依照哪一個方法挑選單變量特徵
        # SelectKBest：選取 K 個最好的特徵，k 為參數，代表你想選擇多少特徵。
        # SelectPercentile：選取多少百分比的特徵，percentile 為參數，代表百分比，用 10 代表 10%。
        
Image(url='http://cumatrixfile.cupoy.com/0000017719EC8070000000036375706F795F72656C656173654349/marathon/1610598311948')

In [10]:
#離散型資料+連續
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
##ValueError: could not convert string to float: 'Male'
#離散型資料要先轉成數值
sex_mapping = {
           'Male': 1,
           'Female': 0}
data['sex1'] = data['sex'].map(sex_mapping)

x=data[['sex1','age', 'height', 'weight']]
display(x)
y=data['insomnia']
x_new = SelectKBest(chi2, k=2).fit_transform(x, y)
x_new.shape
display(x_new)

,sex1,age,height,weight
0,1,23,180,100
1,1,40,170,68
2,1,5,100,20
3,1,30,176,70
4,1,1,70,10
5,0,40,160,45
6,0,16,170,50
7,0,27,166,58
8,0,43,155,58
9,0,8,35,17


array([[180, 100],
       [170,  68],
       [100,  20],
       [176,  70],
       [ 70,  10],
       [160,  45],
       [170,  50],
       [166,  58],
       [155,  58],
       [ 35,  17],
       [170, 101],
       [168,  65],
       [101,  22],
       [175,  79],
       [ 72,  12],
       [163,  40],
       [169,  53],
       [163,  52],
       [151,  56],
       [ 40,  14]])

In [11]:
# 假設用失眠狀態、身高，來預測體重--->體重連續型變數，迴歸問題。
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import chi2
##ValueError: could not convert string to float: 'Male'
#離散型資料要先轉成數值
insomnia_mapping = {
           'Y': 1,
           'N': 0}
data['insomnia1'] = data['insomnia'].map(insomnia_mapping)
sex_mapping = {
           'Male': 1,
           'Female': 0}
data['sex1'] = data['sex'].map(sex_mapping)

x=data[['sex1','age', 'height','insomnia1']]
display(x)
y=data['weight']
x_new = SelectPercentile(chi2, percentile=50).fit_transform(x, y)#top 50%
x_new.shape
display(x_new)

,sex1,age,height,insomnia1
0,1,23,180,1
1,1,40,170,0
2,1,5,100,0
3,1,30,176,0
4,1,1,70,0
5,0,40,160,0
6,0,16,170,1
7,0,27,166,1
8,0,43,155,1
9,0,8,35,0


array([[ 23, 180],
       [ 40, 170],
       [  5, 100],
       [ 30, 176],
       [  1,  70],
       [ 40, 160],
       [ 16, 170],
       [ 27, 166],
       [ 43, 155],
       [  8,  35],
       [ 23, 170],
       [ 39, 168],
       [  5, 101],
       [ 29, 175],
       [  1,  72],
       [ 42, 163],
       [ 13, 169],
       [ 29, 163],
       [ 41, 151],
       [ 10,  40]])

In [12]:
# 包裝法
# 包裝法將特徵選擇看作是搜索問題，根據某一種評量標準，每次選擇某些特徵或排除某些特徵，常用的方法為遞歸特徵消除(RFE)。

In [13]:
from sklearn.datasets import make_friedman1
from sklearn.feature_selection import RFE
from sklearn.svm import SVC
#ValueError: could not convert string to float: 'Male'
#離散要轉換成數值
sex_mapping = {
           'Male': 1,
           'Female': 0}
data['sex1'] = data['sex'].map(sex_mapping)

x=data[['sex1','age', 'height', 'weight']]
display(x)
#insomnia_mapping = {
#           'Y': 1,
#           'N': 0}
#data['insomnia1'] = data['insomnia'].map(insomnia_mapping)

y=data['insomnia1']

estimator = SVC(kernel="linear")
selector = RFE(estimator, n_features_to_select=2, step=1)
selector = selector.fit(x, y)
## True= selected feature
print(selector.support_)
#Selected (i.e., estimated best) features are assigned rank 1.
ranking=selector.ranking_
print(ranking)
rfe_feature = x.loc[:,selector.support_].columns.tolist()
print(rfe_feature)

,sex1,age,height,weight
0,1,23,180,100
1,1,40,170,68
2,1,5,100,20
3,1,30,176,70
4,1,1,70,10
5,0,40,160,45
6,0,16,170,50
7,0,27,166,58
8,0,43,155,58
9,0,8,35,17


[ True False False  True]
[1 2 3 1]
['sex1', 'weight']


# HW

In [14]:
#在鐵達尼資料集中，以 Titanic_train.csv 中，首先將有遺失值的數值刪除，試著將課堂中所學的方法應用上去。
#Q1：目標變數為 Survived，以 Pclass, Age, SibSp, Parch, Fare,Sex, Embarked 為特徵，
#試著用今天教授的包裝法，搭配課程所教的 SVC，試著排出其餘特徵的重要性。

In [15]:
train = pd.read_csv('Titanic_train.csv')
df = train[['Pclass','Age','SibSp','Parch','Fare','Sex','Embarked','Survived']]
df.dropna(axis=0,inplace=True)
df.shape

/Users/mac/opt/anaconda3/envs/regularenv/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


(712, 8)

In [16]:
df.head()

,Pclass,Age,SibSp,Parch,Fare,Sex,Embarked,Survived
0,3,22.0,1,0,7.2500,male,S,0
1,1,38.0,1,0,71.2833,female,C,1
2,3,26.0,0,0,7.9250,female,S,1
3,1,35.0,1,0,53.1000,female,S,1
4,3,35.0,0,0,8.0500,male,S,0


In [17]:
df['Embarked'].unique()

array(['S', 'C', 'Q'], dtype=object)

In [18]:
sex_mapping = {
    'male': 1,
    'female': 0}
Embarker_mapping = {
    'S':2,
    'C':1,
    'Q':0
}
df['Sex1'] = df['Sex'].map(sex_mapping)
df['Embarked1'] = df['Embarked'].map(Embarker_mapping)
df.head()

/Users/mac/opt/anaconda3/envs/regularenv/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/Users/mac/opt/anaconda3/envs/regularenv/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,Pclass,Age,SibSp,Parch,Fare,Sex,Embarked,Survived,Sex1,Embarked1
0,3,22.0,1,0,7.2500,male,S,0,1,2
1,1,38.0,1,0,71.2833,female,C,1,0,1
2,3,26.0,0,0,7.9250,female,S,1,0,2
3,1,35.0,1,0,53.1000,female,S,1,0,2
4,3,35.0,0,0,8.0500,male,S,0,1,2


In [19]:
df.columns

Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex', 'Embarked',
       'Survived', 'Sex1', 'Embarked1'],
      dtype='object')

In [20]:
x=df[['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex1', 'Embarked1']]
display(x.head())
y=df['Survived']
display(y.head())


,Pclass,Age,SibSp,Parch,Fare,Sex1,Embarked1
0,3,22.0,1,0,7.2500,1,2
1,1,38.0,1,0,71.2833,0,1
2,3,26.0,0,0,7.9250,0,2
3,1,35.0,1,0,53.1000,0,2
4,3,35.0,0,0,8.0500,1,2


0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [21]:
estimator = SVC(kernel="linear")
selector = RFE(estimator, n_features_to_select=2, step=1)
selector = selector.fit(x, y)
## True= selected feature
print(selector.support_)
#Selected (i.e., estimated best) features are assigned rank 1.
ranking=selector.ranking_
print(ranking)
rfe_feature = x.loc[:,selector.support_].columns.tolist()
print(rfe_feature)

[False False False  True False  True False]
[2 5 3 1 6 1 4]
['Parch', 'Sex1']


In [22]:
pd.DataFrame({'Rank':ranking,'Features':x.columns}).sort_values('Rank')

,Rank,Features
3,1,Parch
5,1,Sex1
0,2,Pclass
2,3,SibSp
6,4,Embarked1
1,5,Age
4,6,Fare
